In [1]:
import keras.backend
import keras.callbacks
import keras.layers
import keras.models
import keras.optimizers

import matplotlib
matplotlib.use('SVG')

import utils.callbacks
import utils.model_builder
import utils.visualize
import utils.data_provider
import utils.metrics

import skimage.io
import sklearn.metrics

import scipy.stats
import pandas as pd

import tensorflow as tf
import numpy as np

import matplotlib.pyplot as plt

import sys

import os
# Uncomment the following line if you don't have a GPU
#os.environ['CUDA_VISIBLE_DEVICES'] = ''

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/cells2numbers/2017_08_unet/workspace/github/unet4nuclei/unet4nuclei/utils/visualize.py:2: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'svg' by the following code:
  File "/usr/local/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <mo

In [ ]:
# constants
const_lr = 1e-4

tag = '01'

base_dir = "/home/cells2numbers/2017_08_unet/data/"

# Output dirs
out_dir = base_dir + 'unet/experiments/' + tag + '/out'
tb_log_dir = base_dir + "unet/tensorboard/" + tag + "/"
chkpt_dir = base_dir + "unet/experiments/" + tag + "/checkpoints/"

os.makedirs(out_dir, exist_ok=True)
os.makedirs(tb_log_dir, exist_ok=True)
os.makedirs(chkpt_dir, exist_ok=True)

# Files
chkpt_file = chkpt_dir + "{epoch:04d}.hdf5"
csv_log_file = base_dir + "unet/experiments/" + tag + "/log.csv"

# Input dirs
train_dir_x = base_dir + 'unet/split/training/x/'
train_dir_y = base_dir + 'unet/split/training/y/'
val_dir = base_dir + "unet/split/validation/"

# Learning Settings
rescale_labels = True

epochs = 15

batch_size = 10
steps_per_epoch = 500

# make sure these number for to the validation set
val_batch_size = 10
val_steps = int(50 * 4 / val_batch_size)

# generator only params
dim1 = 256
dim2 = 256

bit_depth = 8

In [ ]:
# build session running on GPU 1
configuration = tf.ConfigProto()
#configuration.gpu_options.allow_growth = True
configuration.gpu_options.visible_device_list = "2"
session = tf.Session(config = configuration)

# apply session
keras.backend.set_session(session)

train_gen = utils.data_provider.random_sample_generator(
    train_dir_x,
    train_dir_y,
    batch_size,
    bit_depth,
    dim1,
    dim2,
    rescale_labels
)

val_gen = utils.data_provider.single_data_from_images(
    val_dir + 'x/',
     val_dir + 'y/',
     val_batch_size,
     bit_depth,
     dim1,
     dim2,
     rescale_labels
)

In [ ]:
# build model
model = utils.model_builder.get_model_3_class(dim1, dim2)
model.summary()

loss = "categorical_crossentropy"
metrics = [keras.metrics.categorical_accuracy, 
           utils.metrics.channel_recall(channel=0, name="background_recall"), 
           utils.metrics.channel_precision(channel=0, name="background_precision"),
           utils.metrics.channel_recall(channel=1, name="interior_recall"), 
           utils.metrics.channel_precision(channel=1, name="interior_precision"),
           utils.metrics.channel_recall(channel=2, name="boundary_recall"), 
           utils.metrics.channel_precision(channel=2, name="boundary_precision"),
          ]

optimizer = keras.optimizers.RMSprop(lr = const_lr)

model.compile(loss=loss, metrics=metrics, optimizer=optimizer)

# CALLBACKS
# save model after each epoch
callback_model_checkpoint = keras.callbacks.ModelCheckpoint(
    filepath=chkpt_file,
    save_weights_only=True,
    save_best_only=False
)
callback_csv = keras.callbacks.CSVLogger(filename=csv_log_file)
# callback_splits_and_merges = utils.callbacks.SplitsAndMergesLoggerBoundary(
#     'images',
#     val_gen,
#     gen_calls = val_steps,
#     log_dir=tb_log_dir
# )

callbacks=[callback_model_checkpoint, callback_csv] #, callback_splits_and_merges]


In [ ]:
# TRAIN
statistics = model.fit_generator(
    generator=train_gen,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    validation_data=val_gen,
    validation_steps=val_steps,
    callbacks=callbacks,
    verbose = 1
)

# visualize learning stats
#utils.visualize.visualize_learning_stats_boundary_hard(statistics, out_dir, metrics)

print('Done! :)')